# 获取电影评论数据集

In [1]:
import pyprind

In [2]:
import pandas as pd

In [3]:
import os

In [4]:
import numpy as np

生成一个50000次迭代的进度条

In [5]:
pbar = pyprind.ProgBar(50000) 

In [6]:
labels = {'pos':1,'neg':0}

In [7]:
df = pd.DataFrame()

对语料库文件夹进行遍历，将测试集test\训练集train\正例样本集pos\负例样本集neg导入dataFrame
os.listdir(path) 会返回包含路径path下所有文件的列表
os.path.join(path,file) 会返回一个path和file的组合地址(用/连接)

In [9]:
for s in ('test','train'):
    for l in ('pos','neg'):
        path = '/home/fannian/too_data/aclImdb/%s/%s' % (s,l)
        for file in os.listdir(path):
            with open(os.path.join(path,file), 'r') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()#更新进度条

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:40


In [10]:
df.columns = ['review', 'sentiment']

In [11]:
np.random.seed(0)

np.random.permutation()对数据结构(df)的索引编号进行随机打乱
df.reindex(newindex) 用新的索引编号替换原有索引，若原有索引数据不存在，则为NaN

In [12]:
df = df.reindex(np.random.permutation(df.index))

In [13]:
df.to_csv('/home/fannian/tool_data/outdata/movie_data.csv',index = False)

# 词袋模型-将单词转换为特征向量

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
count = CountVectorizer()#词频：训练后生成一个词频字典

In [16]:
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining and the weather is sweet'
])

In [17]:
bag = count.fit_transform(docs)#生成特征词向量的压缩行稀疏矩阵

In [19]:
print(count.vocabulary_) #返回一个词频字典

{'the': 5, 'sun': 3, 'is': 1, 'shining': 2, 'weather': 6, 'sweet': 4, 'and': 0}


In [24]:
print(bag.toarray()) #返回词向量的array形式

[[0 1 1 1 0 1 0]
 [0 1 0 0 1 1 1]
 [1 2 1 1 1 2 1]]
